# How to use the DefectChemistry Workchain

This notebook will explain the use of the `DefectChemistry` workchain.

In other to select suitable dopants for a materials, one has to consider several dopants possibly in various charge states. `DefectChemistry` workchain is the top most workchain in `AiiDA-defects` package that was designed to make this calculation seamless and easily reproducible. 
Once all the necessary inputs are provided, the workchain will compute all the necessary terms including the chemical potential and the Fermi level in a consistent way without the need for human intervention.
An example of the input file is shown below:


In [ ]:
# Get your normal profile
%load_ext aiida

In [ ]:
%aiida

In [ ]:
import numpy as np
from aiida import orm, engine, common

#Import the workchain 
from aiida_defects.formation_energy.defect_chemistry_qe import DefectChemistryWorkchainQE

In [ ]:
# Set up structures
from pymatgen.core.structure import Structure

pymatgen_structure = Structure.from_file("./Structures/Li3ClO_1x1x1.cif")

# Unitcell
unitcell_structure = orm.StructureData(pymatgen=pymatgen_structure)

# Host 2x2x2 supercell
pymatgen_structure.make_supercell([2,2,2])
host_structure = orm.StructureData(pymatgen=pymatgen_structure)

In [ ]:
# List all possible defects you want to consider when determining the defect chemistry of your material
defect_dict = {
       'V_Li': {'N_site': 3, 'species': {'Li':-1}, 'defect_position': [0.25, 0.0, 0.0], 'charges': [-1.0]},
       'V_O': {'N_site': 1, 'species': {'O':-1}, 'defect_position': [0.0, 0.0, 0.0], 'charges': [2.0]},
       'V_Cl': {'N_site': 1,'species': {'Cl':-1}, 'defect_position': np.array([0.25, 0.25, 0.25]), 'charges': np.array([2.0])},
       'N-O': {'N_site': 1,'species': {'N':1, 'O':-1}, 'defect_position': [0.0, 0.0, 0.0], 'charges': [-1.0]},
       'Mg-Li': {'N_site': 3, 'species': {'Mg':1, 'Li':-1}, 'defect_position': [0.25, 0.0, 0.0], 'charges': [1.0]},
        }

# Formation energies required by the ChemicalPotential workchain
Ef_dict = {
       'intrinsic': {'Li3ClO': -11.075, 'LiClO4': -6.7039, 'LiCl': -4.2545, 'ClO2': -1.6839, 'Cl2O7': -4.0196, 'ClO3': -1.9974, 'Cl2O': -1.1942, 
            'LiO8': -3.6499, 'Li2O': -6.2001, 'Li2O2': -6.6031},
       'N': {'Li3ClO': -11.075, 'LiClO4': -6.7039, 'LiCl': -4.2545, 'Li4NCl': -6.1485, 'NCl3': -0.2089, 'NClO3': -3.6343, 'NClO6': -4.5814, 
           'NClO': -1.5903, 'ClO2': -1.6839, 'Cl2O7': -4.0196, 'ClO3': -1.9974, 'Cl2O': -1.1942, 'LiN3': -1.6777, 'Li3N': -1.8502, 'LiNO3': -7.3655,
           'LiO8': -3.6499, 'Li2O': -6.2001, 'Li2O2': -6.6031, 'N2O5': -5.1059, 'N2O': -1.3466, 'NO2': -2.4734},
        }

In [ ]:
# Codes
pw_code = orm.Code.get_from_string("pw-7.0-qe-eiger")

# Pseudos
pseudo_family = orm.Str('SSSP/1.2/PBEsol/efficiency') # This is the label that was used when installing the pseudos

In [ ]:
# If known, the dielctric constant can be directly provided as below. If not, it can be computed within the workchain by specifying the ph code
dielectric = orm.ArrayData()
dielectric.set_array('epsilon', np.array([[3.12, 0., 0.,], [0., 3.12, 0.], [0., 0., 3.12]]))

# Covariance matrix is needed to construct the gaussian charge model for the correction workchain.
cov_matrix = orm.ArrayData()
cov_matrix.set_array('sigma', np.eye(3))

inputs = {
    'restart_wc' : orm.Bool(False),
    # 'restart_node' : orm.Int(29729), 
    # Structures
    'unitcell' : unitcell_structure,
    'host_structure' : host_structure,
    'defect_info' : orm.Dict(dict=defect_dict),
    # Chemical potential
    'formation_energy_dict' : orm.Dict(dict=Ef_dict),
    'compound' : orm.Str('Li3ClO'),
    'dependent_element' : orm.Str('Li'),
    'ref_energy' : orm.Dict(dict={'Li':-195.51408,'O':-557.49850,'Cl':-451.66500, 'N':-274.00734, 'Mg':-445.18254}),
    'temperature' : orm.Float(300.0),  # in Kelvin
    # Correction scheme
    'correction_scheme' : orm.Str('gaussian'),
    'epsilon' : dielectric, # epsilon_inf = 3.2
    'cutoff' : orm.Float(100.0),
    'charge_model': {
        'model_type': orm.Str('fixed'),
        'fixed':{
            'covariance_matrix': cov_matrix}
        # 'fitted': {
        #     'tolerance': orm.Float(1.0e-3),
        #     'strict_fit': orm.Bool(True),
        #     }
        },
    }


In [ ]:
# Scheduler options
pw_metadata = orm.Dict(dict={ 
    'options': {
        'max_wallclock_seconds': 1*60*60, 
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine':  64, 
            'num_cores_per_mpiproc': 1,  
            }
        }
    })
pw_settings = orm.Dict(dict={'cmdline': ['-nk', '4']})

pw_metadata_unitcell = orm.Dict(dict={
    'options': {
        'max_wallclock_seconds': 1*60*60, 
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine':  16,
            'num_cores_per_mpiproc': 1,
            }
        }
    })
pw_settings_unitcell = orm.Dict(dict={'cmdline': ['-nk', '2']})

dos_code = orm.Code.get_from_string('dos-7.0-qe-eiger')
dos_metadata = orm.Dict( dict={
    'options': {
        'max_wallclock_seconds': 10*60,
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine':  16,
            'num_cores_per_mpiproc': 1,
        }
    },
})

pp_code = orm.Code.get_from_string('pp-7.0-qe-eiger')
pp_metadata = orm.Dict( dict={
#    'description': 'Li3ClO',
    'options': {
        'max_wallclock_seconds': 10*60,
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine':  64,
            'num_cores_per_mpiproc': 1, 
        }
    },
})

# Computational (chosen code is QE)
inputs['qe'] = {
        'dft': {
            'supercell' : {
                'relaxation_scheme': orm.Str('fixed'),
                'code' : pw_code,
                #'kpoints': kpoints,
                'pseudopotential_family': pseudo_family,
                'parameters' : pw_parameters,
                'scheduler_options' : pw_metadata,
                'settings' : pw_settings,
            },
            'unitcell' : {
                'code' : pw_code,
                #'kpoints': kpoints_unitcell,
                'pseudopotential_family': pseudo_family,
                'parameters' : pw_parameters,
                'scheduler_options' : pw_metadata_unitcell,
                'settings' : pw_settings_unitcell,
            }
        },
        'dos' : {
            'code' : dos_code,
            'scheduler_options' : dos_metadata,
        },
#        'dfpt' : {
#            'code' : ph_code,
#            'scheduler_options' : ph_metadata,
#        },
        'pp' : {
            'code' : pp_code,
            'scheduler_options' : pp_metadata,
        }
    }


In [ ]:
workchain_future = engine.submit(DefectChemistryWorkchainQE, **inputs)
print('Submitted workchain with PK=' + str(workchain_future.pk))